In [6]:
%load_ext autoreload
%autoreload 2

import sys
import os
import glob
import time

# torch
import torch
import gc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

gc.collect()
torch.cuda.empty_cache()

# raytune
# from ray import tune
# from ray.tune import CLIReporter
# from ray.tune.schedulers import ASHAScheduler

# sci py stuff
import numpy as np
import matplotlib.pyplot as plt

# set path
deeprad_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

if deeprad_dir not in sys.path:
  sys.path.insert(0, deeprad_dir)


# Import deeprad models
from deeprad.model import Autoencoder, CustomDataSet
from deeprad import utils
import deeprad.traintest_utils as traintest_utils
fd, pp = utils.fd, utils.pp

# Set seeds/device
np.random.seed(2)  # TODO: confirm right location?
device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu'
print("You are using device: %s" % device)

# Data folders
TARGET_FOLDER_PATH = os.path.join(deeprad_dir, "data/traintest/out_data")
CHANNEL_FOLDER_PATH = os.path.join(deeprad_dir, "data/traintest/in_data")
assert os.path.isdir(TARGET_FOLDER_PATH) and os.path.isdir(CHANNEL_FOLDER_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
You are using device: cpu


In [17]:
# Hyperparameters
run_train = False
max_epochs = 2
learning_rate = 1e-3
weight_decay = 1e-5
batch_size = 5
# TODO: SGD/Adam/error

# Model
model_fpath = 'model_{}'.format('epoch_{}_lr_{}'.format(max_epochs, learning_rate))
model_fpath = model_fpath.replace('.', '_')
model_fpath = os.path.join(deeprad_dir, 'models', model_fpath + '.pt')

RUN_TRAIN = True

# Load training/test set
dataset = CustomDataSet(TARGET_FOLDER_PATH, CHANNEL_FOLDER_PATH,
                        transform=transforms.ToTensor(), device=device)
train_size = int(len(dataset) * .8)
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])
iters = train_size // batch_size

print('train size: {}, test size: {}'.format(train_size, test_size))

print('train_size / batch_size = {} / {} = {}'.format(train_size, batch_size, train_size / batch_size))

train size: 1972, test size: 493
train_size / batch_size = 1972 / 5 = 394.4


In [18]:
#---------------------------
# training
#---------------------------
if run_train:
  model = Autoencoder(device)
  model = model.to(device)

  start_time = time.time()
  print("Training {} data, over {} epochs".format(train_size, max_epochs))

  torch.manual_seed(42)
  criterion = nn.MSELoss() # mean square error loss
  optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

  outputs_train = [0] * max_epochs
  train_loss_arr = []

  for epoch in range(max_epochs):
      start = time.time()
      for i, (inputs, labels) in enumerate(train_loader):
          input_batch, label_batch = inputs.to(device), labels.to(device)

          # forward pass
          recon_batch = model.forward(input_batch)
          # calc loss
          loss = criterion(recon_batch, label_batch)

          # calc gradient with autograd
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if i == (iters - 1):
            print('Epoch:{}, Loss:{:.4f}'.format(epoch + 1, loss))
            outputs_train[epoch] = (label_batch, recon_batch)

          # For learning curves
          train_loss_arr.append(loss) 

          del inputs; del label_batch; del recon_batch; del loss; del i
      
  print('time took training {}.'.format(utils.timer(start_time, time.time())))

  # Save Model
  torch.save(model, model_fpath)
  print('Viewing Train Images')


In [26]:
# --------------------------
# testing
#---------------------------

# testing loop
with torch.no_grad():  # to prevent out of memory errors
    model = torch.load(model_fpath, map_location=torch.device('cpu'))
    model.eval()
    torch.manual_seed(42)

    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=batch_size, shuffle=True, drop_last=True)


    outputs_test = [0] * test_size
    test_loss_arr = []
    for input_batch, label_batch in test_loader:
        input_batch = inputs.to(device)
        recon_batch = model(inputs)
        
        loss = criterion(recon_batch, label_batch)
        test_loss_arr.append(loss)
        outputs_test[i] = (inputs, recon)

    # Viz loss 
    _outputs_test = outputs_test[::20]  # 20
    print(len(_outputs_test))

    loss_img_fpath = 'test_loss_' + model_fpath.replace('.pt', '.jpg')
    print('Save img at: {}'.format(loss_img_fpath))
    viz_loss(_outputs_test, img_fpath=loss_img_fpath, img_title='Test Loss')


25
Save img at: test_loss_/mnt/c/Users/saeran2/master/git/deeprad/models/model_epoch_2_lr_0_001.jpg


NameError: name 'clean_img4rad' is not defined

In [ ]:
# loss curves 

f, a = plt.plot()
#train_loss_arr
